In [1]:
import re
import os
import sys
import math
import json
import shutil
import hashlib
import platform
import itertools
import collections
import pkg_resources  # pip install py-rouge
from io import open
from rouge import Rouge
from konlpy.tag import Mecab 

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from tqdm import tqdm
from zipfile import ZipFile

import boto3
from botocore import UNSIGNED
from botocore.client import Config

import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import _LRScheduler
import transformers
from transformers import BartForConditionalGeneration, get_scheduler, PreTrainedTokenizerFast, EarlyStoppingCallback, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM

C:\Users\hist\miniconda3\envs\py38\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Trainer arguments
lr = 1e-3
stop = 3
epoch = 100
batch = 4
seed = 42
device = 'cuda'

# init

In [3]:
class AwsS3Downloader(object):
    def __init__(
        self,
        aws_access_key_id=None,
        aws_secret_access_key=None,
    ):
        self.resource = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
        ).resource("s3")
        self.client = boto3.client(
            "s3",
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            config=Config(signature_version=UNSIGNED),
        )

    def __split_url(self, url: str):
        if url.startswith("s3://"):
            url = url.replace("s3://", "")
        bucket, key = url.split("/", maxsplit=1)
        return bucket, key

    def download(self, url: str, local_dir: str):
        bucket, key = self.__split_url(url)
        filename = os.path.basename(key)
        file_path = os.path.join(local_dir, filename)

        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        meta_data = self.client.head_object(Bucket=bucket, Key=key)
        total_length = int(meta_data.get("ContentLength", 0))

        downloaded = 0

        def progress(chunk):
            nonlocal downloaded
            downloaded += chunk
            done = int(50 * downloaded / total_length)
            sys.stdout.write(
                "\r{}[{}{}]".format(file_path, "█" * done, "." * (50 - done))
            )
            sys.stdout.flush()

        try:
            with open(file_path, "wb") as f:
                self.client.download_fileobj(bucket, key, f, Callback=progress)
            sys.stdout.write("\n")
            sys.stdout.flush()
        except:
            raise Exception(f"downloading file is failed. {url}")
        return file_path

def download(url, chksum=None, cachedir=".cache"):
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    os.makedirs(cachedir_full, exist_ok=True)
    filename = os.path.basename(url)
    file_path = os.path.join(cachedir_full, filename)
    if os.path.isfile(file_path):
        if hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10] == chksum:
            print(f"using cached model. {file_path}")
            return file_path, True

    s3 = AwsS3Downloader()
    file_path = s3.download(url, cachedir_full)
    if chksum:
        assert (
            chksum == hashlib.md5(open(file_path, "rb").read()).hexdigest()[:10]
        ), "corrupted file!"
    return file_path, False

def get_kobart_tokenizer(cachedir=".cache"):
    """Get KoGPT2 Tokenizer file path after downloading"""
    tokenizer = {
        "url": "s3://skt-lsl-nlp-model/KoBART/tokenizers/kobart_base_tokenizer_cased_cf74400bce.zip",
        "chksum": "cf74400bce",
    }
    file_path, is_cached = download(
        tokenizer["url"], tokenizer["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    if (
        not os.path.exists(os.path.join(cachedir_full, "emji_tokenizer"))
        or not is_cached
    ):
        if not is_cached:
            shutil.rmtree(
                os.path.join(cachedir_full, "emji_tokenizer"), ignore_errors=True
            )
        zipf = ZipFile(os.path.expanduser(file_path))
        zipf.extractall(path=cachedir_full)
    tok_path = os.path.join(cachedir_full, "emji_tokenizer/model.json")
    tokenizer_obj = PreTrainedTokenizerFast(
        tokenizer_file=tok_path,
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        mask_token="<mask>",
    )
    return tokenizer_obj

def get_pytorch_kobart_model(ctx="cpu", cachedir=".cache"):
    pytorch_kobart = {
        "url": "s3://skt-lsl-nlp-model/KoBART/models/kobart_base_cased_ff4bda5738.zip",
        "chksum": "ff4bda5738",
    }
    model_zip, is_cached = download(
        pytorch_kobart["url"], pytorch_kobart["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.join(os.getcwd(), cachedir)
    model_path = os.path.join(cachedir_full, "kobart_from_pretrained")
    if not os.path.exists(model_path) or not is_cached:
        if not is_cached:
            shutil.rmtree(model_path, ignore_errors=True)
        zipf = ZipFile(os.path.expanduser(model_zip))
        zipf.extractall(path=cachedir_full)
    return model_path

In [4]:
def make_df(phase):
    work_dir = "C:\\Users\\hist\\Documents\\GitHub\\KoBART"
    if phase == 'train':
        tmp = work_dir+'/022.요약문 및 레포트 생성 데이터/01.데이터/1.Training/라벨링데이터/TL1'
    else:
        tmp = work_dir+'/022.요약문 및 레포트 생성 데이터/01.데이터/2.Validation/라벨링데이터/VL1'
    listdir = os.listdir(tmp)
    df = pd.DataFrame({}, columns = ['genre', 'text', 'label'])
    for i in listdir:
        files = os.listdir(f'{tmp}/{i}/2~3sent')
        for f in tqdm(files):
            with open(f'{tmp}/{i}/2~3sent/{f}', 'r', encoding='utf-8') as json_file:
                j = json.loads(json_file.read())
                df2 = pd.DataFrame.from_dict([{'genre' : i, 
                                               'text'  : j['Meta(Refine)']['passage'], 
                                               'label' : j['Annotation']['summary1']}])
                df = pd.concat([df, df2])
    return df

In [5]:
train = pd.read_csv('train.tsv', sep='\t')
train.columns = ['text', 'label']
train

,text,label
0,일제 강제징용피해자들이 지난 2월 발의된 ‘근로정신대 피해자 지원법률안’의 조속한 ...,13일 광주시의회 시민 소통실에서 시민단체 '근로정신대 할머니와 함께 하는 시민모임...
1,문재인 대통령이 이르면 9일 중폭 개각을 단행할 것으로 보인다. 여권 관계자는 8일...,"문재인 대통령이 이르면 9일 장관 3~4명, 장관급 3명을 교체하는 개각을 단행하고..."
2,전남 어촌마을 정주여건 개선 ‘탄력’ 해수부 공모서 고흥 득량도·강진 신전 등 4개...,"해양수산부 어촌종합개발 거점 개발사업에 전남 고흥 득량도, 강진 신전, 완도 충동,..."
3,여자친구 알몸을 무단으로 촬영해 인터넷 성인 카페에 유포한 남성이 재판에 넘겨졌다....,피해자 A씨는 가해자인 B씨의 엄벌을 촉구하며 국민청원에 피해사실을 공개하였고 결국...
4,대구시의회 배지숙 의장이 개회를 선언하고 있다. 대구시의회(의장 배지숙)는 14일부...,대구시의회가 14일 제1차 본회의를 시작으로 제265회 임시회를 개최하고 ‘대구시시...
...,...,...
34237,"민생 구하기부터 나서야” 자유한국당은 7일 더불어민주당을 향해 ‘재판 불복, 민심 ...",김병준 비상대책위원장은 민주당 이해찬 대표가 대선불복은 절대 받아들일 수 없는 것이...
34238,"민주평화통일자문회의 당진시협의회(회장 김광일, 이하 민주평통) 한성희 수석부회장이 ...","지난 20일, 한성희 수석부회장이 국민훈장을 수상했고, 앞서 지난 19일에는 박병원..."
34239,충남도서관 전경 충남도서관이 한국문화공간건축학회로부터 도서관 부문 '2018 한국문...,"올해로 4회째를 맞는 한국문화공간상에서 충남 도서관이 공공성과 접근성, 사용자들의 ..."
34240,전남도의회는 10일 열린 제 335회 임시회 본회의에서 이광일(더불어민주당·여수 1...,전남도의회는 헌재가 현행 해상경계선을 인정하고 현행 해상경계선을 경계선으로 확정할 ...


In [6]:
# %%time

# train = make_df('train').reset_index(drop=True)
# val = make_df('val').reset_index(drop=True)

In [7]:
# train.to_parquet('train.parquet')
# val.to_parquet('val.parquet')

In [8]:
train = pd.read_parquet('train.parquet')
val   = pd.read_parquet('val.parquet')
# val   = val.sample(n=2, replace=False).reset_index(drop=True)
# train = train[train.text.str.len() < 1024]
# val = val[val.text.str.len() < 1024].sample(n=2, replace=False).reset_index(drop=True)
val

,genre,text,label
0,01.news_r,새해가 밝았다.\n 또 다른 한 해가 시작되었다.\n 눈이 내린 하얀 설원이 앞...,새해라는 미지의 시간에 남길 나의 발자국은 다른 이들에게는 이정표가 될 것이다.
1,01.news_r,"""기다리던 새해가 밝았네요.\n 올해 제가 드디어 아빠가 됩니다.\n "" 국내에서...",공장에서 일하다 왼쪽 팔을 잃은 그는 팔 이식 수술 후 결혼에 성공하여 아빠가 되는...
2,01.news_r,김정은 국무위원장이 지난달 28일부터 31일까지 열린 노동당 중앙위원회 전원회의 ...,노동당 중앙위원회 전원회의 보고에서 북한의 김 국무위원장이 새로운 전략무기를 목격하...
3,01.news_r,"신 의원은 ""총선에 제가 다시 도전한다는 의사표시로 예비후보를 등록했다""고 말했다...",총선에 도전하는 예비후보자들이 선거사무소를 차리고 현수막을 걸자 신 의원도 사람들에...
4,01.news_r,스웨덴에서는 지갑을 몸에 지니지 않아도 일상생활에 불편을 겪지 않는 이들이 있다.\...,스웨덴에서는 지갑을 몸에 지니지 않아도 엄지와 검지 사이의 손등 표면에 마이크로칩을...
...,...,...,...
9145,10.narration,"대자연의 위대함과 생태계의 신비! BBC를 비롯하여, 세계 일류의 다큐멘터리 전문 ...",다큐멘터리 전문 제작자들이 제작한 고급 다큐멘터리를 엄선해 대자연의 위대함과 생태계...
9146,10.narration,"대자연의 위대함과 생태계의 신비! BBC를 비롯하여, 세계 일류의 다큐멘터리 전문 ...",자신들의 영역에서 쫓겨난 할리우드 암사자 무리는 침략자인 방랑자 무리와 대결을 벌이...
9147,10.narration,"대자연의 위대함과 생태계의 신비! \nBBC를 비롯하여, 세계 일류의 다큐멘터리 전...","사막 방울뱀, 웰위치아, 검은등자칼, 코끼리 등은 자원이 턱없이 부족한 아프리카의 ..."
9148,10.narration,역사의 커다란 물줄기가 바뀐 결정적 하루! 역사가 움직인 터닝 포인트를 입체적으로 ...,"역사 토크쇼 역사저널, 그날에서 살수대첩 대승의 기억속에 숨겨졌던 진실들을 파헤친다."


# 전처리

In [9]:
train = train[:-2]
val = train[-2:].reset_index(drop=True)

In [10]:
def preprocss(df):
    df.text = df.text.apply(lambda x : re.sub('\n', ' ',  x))
    df.text = df.text.apply(lambda x : re.sub('[^ .,가-힣A-Za-z0-9]', ' ',  x))
    df.text = df.text.apply(lambda x : re.sub(' +', ' ',  x).strip())
    return df

train = preprocss(train)
val = preprocss(val)

In [11]:
val.loc[0, 'text']

'다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.'

In [12]:
val.loc[0, 'label']

'인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.'

In [13]:
train.text.str.len().min()

195

In [14]:
# train = train[(train.text.str.len() < 510)].reset_index(drop=True)
# val = val[(val.text.str.len() < 510)].reset_index(drop=True)[:2]
# val

# Dataset

In [15]:
class KoBARTSumDataset(Dataset):
    def __init__(self, df, tokenizer, max_len, ignore_index=-100):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.df = df
        self.len = len(self.df)

        self.pad_index = self.tokenizer.pad_token_id
        self.ignore_index = ignore_index

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]

        return inputs
    
    def __getitem__(self, idx):
        instance = self.df.iloc[idx]
        input_ids = self.tokenizer.encode(instance['text'])
        input_ids = np.append(input_ids, self.tokenizer.eos_token_id)
        input_ids = self.add_padding_data(input_ids)
        input_ids = np.insert(input_ids, 0 , self.tokenizer.bos_token_id)
        
        label_ids = self.tokenizer.encode(instance['label'])
        label_ids.append(self.tokenizer.eos_token_id)
        label_ids.insert(0, self.tokenizer.bos_token_id)
        
        dec_input_ids = [self.tokenizer.eos_token_id]
        dec_input_ids += label_ids[:-1]
        dec_input_ids = self.add_padding_data(dec_input_ids)
        label_ids = self.add_ignored_data(label_ids)
        
        input_ids = torch.tensor(np.array(input_ids)).long()
        decoder_input_ids = torch.tensor(np.array(dec_input_ids)).long()
        
        return {'input_ids': input_ids,
                'attention_mask': input_ids.ne(self.tokenizer.pad_token_id).float(),
                'decoder_input_ids': decoder_input_ids,
                'decoder_attention_mask' : decoder_input_ids.ne(self.tokenizer.pad_token_id).float(),
                'labels': np.array(label_ids, dtype=np.int_)}
        
        # return {'input_ids': np.array(input_ids, dtype=np.int_),
        #         'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
        #         'labels': np.array(label_ids, dtype=np.int_)}

    def __len__(self):
        return self.len
train_dataset = KoBARTSumDataset(train, PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1'), 512)
val_dataset = KoBARTSumDataset(val, PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1'), 512)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [16]:
train_dataset[-1]

{'input_ids': tensor([    0, 14029, 12060, 19753, 14064,  9776, 20010, 17851, 17304, 29664,
         15085, 15571, 10443, 14956, 24346, 14370, 14055, 17760, 28394, 20488,
         14582, 15269, 17813, 27446, 18347, 28394, 20488, 14282, 14768, 22396,
         14199, 17270, 14166, 11042, 14330, 14895, 15758, 16993, 15630, 14049,
         14114, 17436, 14949,  9171,   245, 14538, 20466, 15780, 20037, 16487,
         22161, 12070, 15813, 14150, 14169, 12293, 14129,   243, 14034, 15934,
         29352, 27722, 14270, 20148,  9754,  9738, 21686, 14282, 15166, 10888,
         14130, 29574, 14655, 18262, 14106, 19839, 14323, 28236, 16795, 23438,
         20137, 14130, 14693, 15813,   243, 26321, 10834, 14293, 17297, 12798,
         14150,   243, 14174,   243, 14031,  9754, 11229, 11863, 14293, 28825,
           243, 14850, 19083, 14456,  9866, 10496, 10806,  9120, 14293, 14240,
         12798, 10428,   243, 20212, 14219, 11986, 10428, 12037, 14169, 12293,
         14129,   245, 14317, 12007, 29

# Model

In [17]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v1')
        self.model.train()
        self.bos_token = '<s>'
        self.eos_token = '</s>'
        
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')
        self.pad_token_id = self.tokenizer.pad_token_id
        
    def forward(self, inputs):
        # attention_mask = inputs['input_ids'].ne(self.pad_token_id).float()
        # decoder_attention_mask = inputs['decoder_input_ids'].ne(self.pad_token_id).float()
        
        # return self.model(input_ids=inputs['input_ids'],
        #                   attention_mask=attention_mask,
        #                   decoder_input_ids=inputs['decoder_input_ids'],
        #                   decoder_attention_mask=decoder_attention_mask,
        #                   labels=inputs['labels'], return_dict=True)
        return self.model(input_ids=inputs['input_ids'],
                  attention_mask=inputs['attention_mask'],
                  decoder_input_ids=inputs['decoder_input_ids'],
                  decoder_attention_mask=inputs['decoder_attention_mask'],
                  labels=inputs['labels'], return_dict=True)

In [18]:
model = AutoModelForSeq2SeqLM.from_pretrained('gogamza/kobart-base-v1')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v1')

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [19]:
collator = DataCollatorForSeq2Seq(tokenizer, model=model, label_pad_token_id=tokenizer.pad_token_id)

In [20]:
rouge = Rouge()
def compute_metrics(pred):
    preds, labels = pred

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    print('Text\n', val.text[0])
    print(f'Summarize\nGold : {labels[0]}\nGen : {preds[0]}')
    
    labels = ['\n'.join(labels)]
    preds = ['\n'.join(preds)]
    score = rouge.get_scores(preds, labels, avg=True)
    return {
        "ROUGE-1" : score['rouge-1'],
        "ROUGE-2" : score['rouge-2'],
        "ROUGE-L" : score['rouge-l'],
    }

In [21]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
# scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=1, eta_min=0, last_epoch=-1)

lr_scheduler = transformers.get_cosine_schedule_with_warmup(optimizer=optimizer, 
                                                            num_warmup_steps=100, 
                                                            num_training_steps=epoch * len(train_dataset) * batch, 
                                                            last_epoch = -1)

In [ ]:
args = Seq2SeqTrainingArguments(run_name = f'KoBARTSum',
                                output_dir= f"models",
                                evaluation_strategy="steps",
                                eval_steps=100,
                                save_steps=100,
                                logging_steps=100,
                                save_total_limit = 2,
                                
                                per_device_train_batch_size=batch,
                                per_device_eval_batch_size=batch,
                                gradient_accumulation_steps=16,
                                num_train_epochs=epoch,
                                
                                load_best_model_at_end=True,
                                fp16=True,
                                do_train=True,
                                do_eval=True,
                                predict_with_generate=True,)

trainer = Seq2SeqTrainer(model=model,
                         tokenizer=tokenizer,
                         args=args,

                         train_dataset=train_dataset,
                         eval_dataset=val_dataset,

                         compute_metrics=compute_metrics,
                         optimizers=(optimizer, lr_scheduler),
                         # callbacks=[EarlyStoppingCallback(early_stopping_patience=stop)],
                         data_collator=collator,)
trainer.train()

Using cuda_amp half precision backend
***** Running training *****
  Num examples = 73338
  Num Epochs = 100
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 16
  Total optimization steps = 114500
  Number of trainable parameters = 123859968
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Rouge-1,Rouge-2,Rouge-l
100,3.291500,1.540322,"{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}"
200,1.564300,1.314538,"{'r': 0.47058823529411764, 'p': 0.42105263157894735, 'f': 0.44444443945987655}","{'r': 0.375, 'p': 0.3333333333333333, 'f': 0.35294117148788934}","{'r': 0.47058823529411764, 'p': 0.42105263157894735, 'f': 0.44444443945987655}"
300,1.492100,1.235838,"{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}","{'r': 0.0, 'p': 0.0, 'f': 0.0}"
400,1.445100,1.189054,"{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}","{'r': 0.4375, 'p': 0.4375, 'f': 0.43749999500000003}","{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}"
500,1.418900,1.216791,"{'r': 0.47058823529411764, 'p': 0.4444444444444444, 'f': 0.4571428521469388}","{'r': 0.375, 'p': 0.35294117647058826, 'f': 0.363636358640955}","{'r': 0.47058823529411764, 'p': 0.4444444444444444, 'f': 0.4571428521469388}"
600,1.401000,1.263261,"{'r': 0.4117647058823529, 'p': 0.3888888888888889, 'f': 0.3999999950040816}","{'r': 0.3125, 'p': 0.29411764705882354, 'f': 0.30303029803489445}","{'r': 0.4117647058823529, 'p': 0.3888888888888889, 'f': 0.3999999950040816}"
700,1.377100,1.202939,"{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}","{'r': 0.4375, 'p': 0.4375, 'f': 0.43749999500000003}","{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}"
800,1.362000,1.157833,"{'r': 0.5294117647058824, 'p': 0.47368421052631576, 'f': 0.4999999950154322}","{'r': 0.375, 'p': 0.3333333333333333, 'f': 0.35294117148788934}","{'r': 0.5294117647058824, 'p': 0.47368421052631576, 'f': 0.4999999950154322}"
900,1.359600,1.220365,"{'r': 0.23529411764705882, 'p': 0.2222222222222222, 'f': 0.2285714235755103}","{'r': 0.1875, 'p': 0.17647058823529413, 'f': 0.18181817682277332}","{'r': 0.23529411764705882, 'p': 0.2222222222222222, 'f': 0.2285714235755103}"
1000,1.343600,1.148808,"{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}","{'r': 0.4375, 'p': 0.4375, 'f': 0.43749999500000003}","{'r': 0.6470588235294118, 'p': 0.6470588235294118, 'f': 0.6470588185294118}"


***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



Text
 다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.
Summarize
Gold : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.
Gen : 다큐 온은 시청자들이 필요로 하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고


Saving model checkpoint to models\checkpoint-100
Configuration saved in models\checkpoint-100\config.json
Configuration saved in models\checkpoint-100\generation_config.json
Model weights saved in models\checkpoint-100\pytorch_model.bin
tokenizer config file saved in models\checkpoint-100\tokenizer_config.json
Special tokens file saved in models\checkpoint-100\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-1000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



...

Saving model checkpoint to models\checkpoint-13000
Configuration saved in models\checkpoint-13000\config.json
Configuration saved in models\checkpoint-13000\generation_config.json
Model weights saved in models\checkpoint-13000\pytorch_model.bin
tokenizer config file saved in models\checkpoint-13000\tokenizer_config.json
Special tokens file saved in models\checkpoint-13000\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-12900] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



Text
 다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.
Summarize
Gold : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.
Gen : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.


Saving model checkpoint to models\checkpoint-13100
Configuration saved in models\checkpoint-13100\config.json
Configuration saved in models\checkpoint-13100\generation_config.json
Model weights saved in models\checkpoint-13100\pytorch_model.bin
tokenizer config file saved in models\checkpoint-13100\tokenizer_config.json
Special tokens file saved in models\checkpoint-13100\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-13000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2
  Batch size = 4
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 1,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "pad_token_id": 3,
  "transformers_version": "4.26.0"
}



Text
 다큐 온 은 이 시대가 요구하는 모든 온 이야기를, 갓 지은 밥처럼 따뜻하고 영양이 풍부하게 만들어 온 , 시청자들이 필요로 하는 바로 그 시점에 ON 전달한다. 탄소중립시대, 우리가 나무를 심는 이유. 인간은 단순히 숨 쉬는 것만으로도 이산화탄소를 배출하고 있다. 과연 그 양은 얼마나 될까 실험을 통해 살펴본 결과 001시간 기준 약 27g. 인간이 80세까지 배출하는 양은 약 20여 톤. 이 탄소를 상쇄하려면 소나무 160그루가 필요하다. 그러나 인간이 일상생활을 영위하는 과정에는 무수히 많은 탄소가 배출된다. 의식주를 포함한 모든 생활 속에서 전기를 사용해야 하는데, 이 에너지의 대부분은 화석 연료에 의한 발전으로 가능하기 때문이다. 기본적인 생활 중에 배출되는 탄소발자국을 따라가 본다.
Summarize
Gold : 인간은 80세까지 20여 톤의 이산화탄소를 배출하고 있다.
Gen : 인간은 80세까지 20여 톤의 이산화탄소를 배출한다.


Saving model checkpoint to models\checkpoint-13200
Configuration saved in models\checkpoint-13200\config.json
Configuration saved in models\checkpoint-13200\generation_config.json
Model weights saved in models\checkpoint-13200\pytorch_model.bin
tokenizer config file saved in models\checkpoint-13200\tokenizer_config.json
Special tokens file saved in models\checkpoint-13200\special_tokens_map.json
Deleting older checkpoint [models\checkpoint-13100] due to args.save_total_limit


In [ ]:
torch.tensor(train_dataset[0]['input_ids']).to(device)

In [ ]:
out = model.generate(input_ids = torch.tensor(train_dataset[0]['input_ids']).to(device)[None,:])

In [ ]:
def infer(text):
    if type(text) == int:
        out = model.generate(input_ids = torch.tensor(train_dataset[text]['input_ids'])[None,:].to(device)).logits.argmax(2)
        result = tokenizer.decode(out[0])
    else:
        tmp = [0] + tokenizer.encode(text)+[1]
        out = model.generate(input_ids = torch.tensor(tmp[None,:].to(device)).logits.argmax(2))
        result = tokenizer.decode(out[0])
    return print(result)
                             

In [ ]:
model.forward()

In [ ]:
text = ''
infer(text)

In [ ]:
tokenizer.decode(np.where(train_dataset[0]['labels']>=0, train_dataset[0]['labels'], 1))

In [ ]:
tokenizer.decode(out[0])

In [ ]:
out.logits.argmax(2)

In [ ]:
tmp = '아파트 분양시장이 실수요자 중심으로 바뀌면서 초역세권 입지와 변화하는 라이프스타일에 맞춘 혁신평면이 아파트 선택에 미치는 영향력이 커지고 있는 가운데, 태왕이 지난 22일 공개한 ‘성당 태왕아너스 메트로’ 모델하우스를 찾은 방문객들은 합리적인 분양가와 중도금무이자 등의 분양조건도 실수요자에게 유리해 높은 청약경쟁률을 기대했다.'
tmp = [0] + tokenizer.encode(tmp) + [1]

In [ ]:
out = model.generate(input_ids = torch.tensor(tmp)[None,:].to(device))

In [ ]:
tokenizer.decode(out[0])